<img src="https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2Fe88ebbacb848b09e477d11eedf4209d10ea4ac0a-1399x537.png&w=3840&q=75">


# <b><span style='color:#4CAF50'>|</span> Mission: <span style='color:#4CAF50'>How does Vector Databases work?</span><span style='color:#FF5733'>

***

**Please consider upvoting the notebook if you found it useful** 

Welcome, Kaggle enthusiasts and AI pioneers! In the last year, we have witnessed an unprecedented boom in technology, particularly in the realm of Large Language Models (LLMs) and broader Artificial Intelligence (AI) systems. Tools like ChatGPT have not just entered the mainstream but have revolutionized it, reshaping how we gather and process information across various sectors.

In this notebook, we dive into the heart of this technological revolution by focusing on a specific aspect of AI - vector databases, with a special emphasis on Pinecone. You will embark on a journey to understand and master the use of Pinecone for similarity search and data retrieval. By leveraging advanced vector database techniques, we aim to equip you with the skills and knowledge to harness the full potential of vector databases in AI applications.
    
Hope you enjoy it and find it useful.
    

### <b><span style='color:#FF5733'>Table of Contents</span></b> <a class='anchor' id='top'></a>
<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
<li> <a href="#install_libraries">Install libraries</a></li>
<li><a href="#import_libraries">Import Libraries</a></li>
<li><a href="#initialization">Initialization of Pinecone Client</a></li>
<li><a href="#Load_model">Load SentenceTransformer Model</a></li>
<li><a href="#load_data">Load Custom Dataset</a></li>
<li><a href="#create_emb">Make Embeddings</a></li>
<li><a href="#Search">Query Search</a></li>
</div>




# <b><span style='color:#4CAF50'>|</span> Install Libraries</b><a class='anchor' id='install_libraries'></a> [↑](#top) 

***

Install all the required libraries for this notebook.

In [12]:
!pip install -U sentence-transformers
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# <b><span style='color:#4CAF50'>|</span> Import Libraries</b><a class='anchor' id='import_libraries'></a> [↑](#top) 

***

Import all the required libraries for this notebook.

In [13]:
import os
import time
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec, PodSpec

# <b><span style='color:#4CAF50'>|</span> Initializing Pinecone Index</b><a class='anchor' id='initialization'></a> [↑](#top) 

***
In this step, we create our Pinecone client using api_key and initializing pinecone index. Note - api_key can be obtained after log in to pinecone. 

In [14]:
# Check if running in serverless mode
use_serverless = False

# Configure Pinecone client
api_key = os.environ.get('PINECONE_API_KEY')
environment = os.environ.get('PINECONE_ENVIRONMENT', 'gcp-starter')
pc = Pinecone(api_key=api_key)

# Define Pinecone index specifications
if use_serverless:
    spec = ServerlessSpec(cloud='gcp', region='asia-southeast1-gcp')
else:
    spec = PodSpec(environment=environment)
    
# Define Pinecone index name
index_name = "dataset"

if index_name not in pc.list_indexes().names():
    dimensions = 384 # We have use e5-small transformer model which generates embeddings of dimension 384
    pc.create_index(
        name=index_name,
        dimension=dimensions,
        metric="cosine",
        spec=spec
    )
# Wait until index is ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

# <b><span style='color:#4CAF50'>|</span> Load SentenceTransformer Model</b><a class='anchor' id='Load_model'></a> [↑](#top) 

***

Here, we load pretrained SentenceTransfomer model for generating embeddings of data.

In [15]:
# Load SentenceTransformer model
model = SentenceTransformer('intfloat/e5-small')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# <b><span style='color:#4CAF50'>|</span> Load Custom dataset</b><a class='anchor' id='load_data'></a> [↑](#top) 

***

Text dataset with pre-defined column names is to be loaded here.

In [16]:
path = '/kaggle/input/health-data/Healthcare.csv'
data = pd.read_csv(path)
data['id'] = data.index.astype(str)

In [17]:
data.columns   # Custom data must contain the following column names

Index(['Unnamed: 0', 'Query', 'Answer', 'Metadata1', 'Metadata2', 'id'], dtype='object')

# <b><span style='color:#4CAF50'>|</span> Create text embeddings</b><a class='anchor' id='create_emb'></a> [↑](#top) 

***

In this process, we created text embeddings derived from the Query column of our dataset and performed upserts into a Pinecone index. These embeddings enable efficient retrieval of the most similar queries when users perform searches, allowing access to metadata associated with those queries. Given the constraint of upserting only up to 1000 vectors into a Pinecone index, we manage this by batching the upsert operations in chunks of 1000 queries.

In [ ]:
namespace = 'dataspace'

input_texts = data['Query']

for chunk_start in range(0, len(data), 1000):
    chunk_end = min(chunk_start + 1000, len(data))
    chunk = data.iloc[chunk_start:chunk_end]
    emb = [model.encode(query, normalize_embeddings=True) for query in chunk['Query']]
    chunk['embedding'] = emb
    index.upsert(vectors=zip(chunk['id'], chunk.embedding),  namespace=namespace)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.16412,
 'namespaces': {'dataspace': {'vector_count': 16412}},
 'total_vector_count': 16412}

# <b><span style='color:#4CAF50'>|</span>Query Search</b><a class='anchor' id='Search'></a> [↑](#top) 

***

Here, user can enter query.

In [22]:
q = input("Enter your query about the data:")

vec = model.encode(q)
result = index.query(namespace='dataspace',
    vector=vec.tolist(),
            top_k=5,
            include_values=False
)
for res in result['matches']:
    print(res)

id = result['matches'][0]['id']


Enter your query about the data: What is diabetes?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'id': '110', 'score': 0.988319039, 'values': []}
{'id': '103', 'score': 0.988319039, 'values': []}
{'id': '118', 'score': 0.988319039, 'values': []}
{'id': '109', 'score': 0.988319039, 'values': []}
{'id': '111', 'score': 0.988319039, 'values': []}


In [24]:
answer = data[data['id'] == id]['Answer'].values[0]
print(answer)

Prediabetes means your blood glucose levels are higher than normal but not high enough for a diagnosis of diabetes. In 2012, about 86 million people in the U.S. had prediabetes, and 51% of those 65 or older had prediabetes. People with prediabetes are at an increased risk for developing type 2 diabetes and for heart disease and stroke. The good news is that if you have prediabetes, you can reduce your risk of getting type 2 diabetes. With modest weight loss and moderate physical activity, you can delay or prevent type 2 diabetes.  Learn more about prediabetes here.


In [25]:
index.delete(deleteAll=True, namespace = 'dataspace')#To delete all data store in pinecone index with particular namespace

{}